In [1]:
import torch, torchvision, os, PIL, pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

/home/joseph/miniconda3/envs/multimodal/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/joseph/miniconda3/envs/multimodal/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def show(tensor, num=25, wandb=0, name=''):
    data = tensor.detach().cpu()
    grid = make_grid(data[:num], nrow=5).permute(1,2,0)
    
    ## optional
    if (wandb==1 and wandbact==1): # wandb : weights and biases (online tool)
        wandb.log({name:wandb.Image(grid.numpy().clip(0,1))})
    
    plt.imshow(grid.clip(0,1)) # clip the values of the pixles to be inbetween 0~1
    plt.show()
    
### hyperparameters and general parameters
n_epochs = 10000
batch_size = 128
lr = 1e-4
z_dim = 200
device='cuda'

cur_step = 0
crit_cycles = 5 # 5 cycles of the critic -> 1 cycle of the generator training for advanced Wesserstein GAN
gen_losses = []
crit_losses = []
show_step = 35
save_step = 35 # will be checkpoint

wandbact = 1 # to track stats through weights and biases, optional


In [ ]:
### optional
!pip install wandb -qqq
import wandb
import os
from dotenv import load_dotenv
load_dotenv()
wandb_api_key = os.getenv("WANDB_API_KEY")
wandb.login(key=wandb_api_key)